In [1]:
import requests
import json
import datetime
import pandas as pd
import re
import os
import random
import urllib3
import time
import matplotlib
import nltk

from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from datetime import datetime
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

path = '../data/TW/'

# 0. load all data

## read in LTN News

In [2]:
ltn_df = pd.read_csv(path + 'raw/ltn_news.csv')

ltn_contents_df = pd.read_csv(path + 'raw/ltn_contents.csv')

In [3]:
ltn_df = pd.merge(ltn_df, ltn_contents_df, on="id")
ltn_df.shape

(6416, 10)

In [4]:
ltn_df.head(3)

,id,url,title,release_at,keyword_dpp,keyword_han,keyword_kmt,keyword_tsai,index,content
0,3043716,https://news.ltn.com.tw/news/politics/breaking...,台中想吃雞排看這裡 2500份19、20日手刀排一波,2020/01/17,0,0,0,2,6,館長兌現承諾發送雞排，其中2000份20日下午3點在陳柏惟烏日新興路481號服務處發放。（記...
1,3043404,https://news.ltn.com.tw/news/politics/breaking...,英國駐台代表晉見蔡總統 恭賀台灣蓬勃發展的民主機制,2020/01/17,0,0,0,2,0,總統蔡英文（右）17日上午在總統府接見英國在台辦事處代表唐凱琳（Catherine Nett...
2,3043321,https://news.ltn.com.tw/news/politics/breaking...,與國外知名網紅聊女權、寵物 蔡總統：台灣重能力不看性別,2020/01/17,0,0,0,2,1,總統蔡英文與百萬網紅塔米爾暢談各項議題。（中央社） 〔即時新聞／綜合報導〕總統蔡英文近期接受...


In [5]:
print(ltn_df.shape)
ltn_df = ltn_df.dropna(axis=0)
ltn_df = ltn_df.drop(columns=['id', 'url'])
print(ltn_df.shape)

(6416, 10)
(6416, 8)


In [6]:
## create onehot coded keywords recors
ltn_df['keyword_tsai'] = False
ltn_df.loc[ltn_df.content.str.contains('蔡英文', na=False), 'keyword_tsai'] = True

ltn_df['keyword_dpp'] = False
ltn_df.loc[ltn_df.content.str.contains('民進黨', na=False), 'keyword_dpp'] = True

ltn_df['keyword_han'] = False
ltn_df.loc[ltn_df.content.str.contains('韓國瑜', na=False), 'keyword_han'] = True

ltn_df['keyword_kmt'] = False
ltn_df.loc[ltn_df.content.str.contains('國民黨', na=False), 'keyword_kmt'] = True

ltn_df['keyword_election'] = False
ltn_df.loc[ltn_df.content.str.contains('|'.join(['總統大選', '選舉', "大選"]), na=False), 'keyword_election'] = True

print(ltn_df.shape)
ltn_df = ltn_df[ltn_df[['keyword_tsai', 'keyword_han', 'keyword_kmt', 'keyword_dpp', 'keyword_election']].any(axis=1)]
print(ltn_df.shape)

ltn_df.head(2)


(6416, 9)
(6387, 9)


,title,release_at,keyword_dpp,keyword_han,keyword_kmt,keyword_tsai,index,content,keyword_election
0,台中想吃雞排看這裡 2500份19、20日手刀排一波,2020/01/17,False,False,False,True,6,館長兌現承諾發送雞排，其中2000份20日下午3點在陳柏惟烏日新興路481號服務處發放。（記...,True
1,英國駐台代表晉見蔡總統 恭賀台灣蓬勃發展的民主機制,2020/01/17,False,False,False,True,0,總統蔡英文（右）17日上午在總統府接見英國在台辦事處代表唐凱琳（Catherine Nett...,True


In [7]:
ltn_df['date'] = pd.to_datetime(ltn_df["release_at"], utc=True)
ltn_df = ltn_df.drop(columns=['release_at'])
ltn_df.head()

,title,keyword_dpp,keyword_han,keyword_kmt,keyword_tsai,index,content,keyword_election,date
0,台中想吃雞排看這裡 2500份19、20日手刀排一波,False,False,False,True,6,館長兌現承諾發送雞排，其中2000份20日下午3點在陳柏惟烏日新興路481號服務處發放。（記...,True,2020-01-17 00:00:00+00:00
1,英國駐台代表晉見蔡總統 恭賀台灣蓬勃發展的民主機制,False,False,False,True,0,總統蔡英文（右）17日上午在總統府接見英國在台辦事處代表唐凱琳（Catherine Nett...,True,2020-01-17 00:00:00+00:00
2,與國外知名網紅聊女權、寵物 蔡總統：台灣重能力不看性別,False,False,False,True,1,總統蔡英文與百萬網紅塔米爾暢談各項議題。（中央社） 〔即時新聞／綜合報導〕總統蔡英文近期接受...,False,2020-01-17 00:00:00+00:00
3,酈英傑：美台盼締造共同未來 是「真朋友真進展」,False,False,False,True,2,美國在台協會與外交部17日舉辦「數位對話公共論壇：增進美台人民的關係」，美國在台協會處長酈英...,False,2020-01-17 00:00:00+00:00
4,台美瓜宏四方會談 吳釗燮：促進區域合作及發展,False,False,False,True,3,外交部長吳釗燮今在推特上推文表示，台美瓜宏舉行四方會談，推文照片並清楚可見四國國旗。（取自外...,False,2020-01-17 00:00:00+00:00


In [8]:
ltn_df['source'] = 'ltn'

## read in CTI news

In [9]:
cti_df =pd.read_csv(path +"raw/cti_news.csv")
print(cti_df.shape)
cti_df.head(2)

(4632, 9)


,title,release_at,news_type.name,keyword_dpp,keyword_han,keyword_kmt,keyword_tsai,url,content
0,蔡英文接見英駐台代表唐凱琳 盼深化台英關係,2020-01-17 18:01:01,政治要聞,0,0,0,1,https://ctinews.com/news/items/D0xdO8jBaY,總統蔡英文今日接見英國在台辦事處代表唐凱琳(Catherine Nettleton)，她重...
1,蔡英文當選第6天 LSE校長詭異未發賀電,2020-01-17 17:01:01,政治要聞,0,0,0,1,https://ctinews.com/news/items/kYaDLKR9a9,1月11日總統蔡英文高票連任至今已經6天，相較於英國和美國外交部長立即發出賀電，蔡的母校倫...


In [10]:
print(cti_df.shape)
cti_df = cti_df.dropna(axis=0)
print(cti_df.shape)
cti_df.head(2)

(4632, 9)
(4283, 9)


,title,release_at,news_type.name,keyword_dpp,keyword_han,keyword_kmt,keyword_tsai,url,content
0,蔡英文接見英駐台代表唐凱琳 盼深化台英關係,2020-01-17 18:01:01,政治要聞,0,0,0,1,https://ctinews.com/news/items/D0xdO8jBaY,總統蔡英文今日接見英國在台辦事處代表唐凱琳(Catherine Nettleton)，她重...
1,蔡英文當選第6天 LSE校長詭異未發賀電,2020-01-17 17:01:01,政治要聞,0,0,0,1,https://ctinews.com/news/items/kYaDLKR9a9,1月11日總統蔡英文高票連任至今已經6天，相較於英國和美國外交部長立即發出賀電，蔡的母校倫...


In [11]:
cti_df['date'] = pd.to_datetime(cti_df["release_at"], utc=True)
cti_df = cti_df.drop(columns=['release_at', 'news_type.name','url'])
cti_df.head()

,title,keyword_dpp,keyword_han,keyword_kmt,keyword_tsai,content,date
0,蔡英文接見英駐台代表唐凱琳 盼深化台英關係,0,0,0,1,總統蔡英文今日接見英國在台辦事處代表唐凱琳(Catherine Nettleton)，她重...,2020-01-17 18:01:01+00:00
1,蔡英文當選第6天 LSE校長詭異未發賀電,0,0,0,1,1月11日總統蔡英文高票連任至今已經6天，相較於英國和美國外交部長立即發出賀電，蔡的母校倫...,2020-01-17 17:01:01+00:00
2,感恩餐會 蔡英文：逆風拚到逆轉 就是民進黨精神,0,0,0,1,蔡英文總統今出席民進黨感恩餐會，場面輕鬆，蔡英文致詞時表示，「今天要好好吃一頓，才能為台...,2020-01-17 16:01:02+00:00
3,首位原住民上將徐衍璞授階 蔡英文：國土主權寸步不讓,0,0,0,1,總統蔡英文今（17）日主持「國軍重要高階幹部晉任布達授階典禮」時表示，近年中共持續透過外交...,2020-01-17 14:01:01+00:00
5,蔡英文視察國軍：最大的責任是提升軍人尊嚴,0,0,0,1,總統蔡英文今（16）日至桃園視導陸軍機步269旅，感謝國軍在春節期間堅守崗位，戮力戰訓本務...,2020-01-16 17:01:02+00:00


In [12]:
## create onehot coded keywords recors
cti_df['keyword_tsai'] = False
cti_df.loc[cti_df.content.str.contains('蔡英文', na=False), 'keyword_tsai'] = True

cti_df['keyword_dpp'] = False
cti_df.loc[cti_df.content.str.contains('民進黨', na=False), 'keyword_dpp'] = True

cti_df['keyword_han'] = False
cti_df.loc[cti_df.content.str.contains('韓國瑜', na=False), 'keyword_han'] = True

cti_df['keyword_kmt'] = False
cti_df.loc[cti_df.content.str.contains('國民黨', na=False), 'keyword_kmt'] = True

cti_df['keyword_election'] = False
cti_df.loc[cti_df.content.str.contains('|'.join(['總統大選', '選舉', "大選"]), na=False), 'keyword_election'] = True

print(cti_df.shape)
cti_df = cti_df[cti_df[['keyword_tsai', 'keyword_han', 'keyword_kmt', 'keyword_dpp', 'keyword_election']].any(axis=1)]
print(cti_df.shape)

cti_df.head(2)


(4283, 8)
(4269, 8)


,title,keyword_dpp,keyword_han,keyword_kmt,keyword_tsai,content,date,keyword_election
0,蔡英文接見英駐台代表唐凱琳 盼深化台英關係,False,False,False,True,總統蔡英文今日接見英國在台辦事處代表唐凱琳(Catherine Nettleton)，她重...,2020-01-17 18:01:01+00:00,True
1,蔡英文當選第6天 LSE校長詭異未發賀電,False,False,False,True,1月11日總統蔡英文高票連任至今已經6天，相較於英國和美國外交部長立即發出賀電，蔡的母校倫...,2020-01-17 17:01:01+00:00,False


In [13]:
cti_df['source'] = 'cti'

## read in central positon (cna pts reporter)

In [14]:
cna_df = pd.read_csv(path + 'raw/cna.csv')
print(cna_df.shape)
reporter_df = pd.read_csv(path + 'raw/reporter.csv')
print(reporter_df.shape)

pts_df = pd.read_csv(path + 'raw/pts.csv')
print(pts_df.shape)

neu_df = pd.concat([cna_df, reporter_df]).reset_index()
neu_df = pd.concat([neu_df, pts_df]).reset_index()
print(neu_df.shape)

(1829, 8)
(125, 7)
(179, 8)
(2133, 10)


In [15]:
neu_df["content"] = neu_df.content.str.replace('請同意我們的隱私權規範，才能啟用聽新聞的功能。', '')
neu_df["content"] = neu_df.content.str.replace('\n\n', ' ')

In [16]:
## create onehot coded keywords recors
neu_df['keyword_tsai'] = False
neu_df.loc[neu_df.content.str.contains('蔡英文', na=False), 'keyword_tsai'] = True

neu_df['keyword_dpp'] = False
neu_df.loc[neu_df.content.str.contains('民進黨', na=False), 'keyword_dpp'] = True

neu_df['keyword_han'] = False
neu_df.loc[neu_df.content.str.contains('韓國瑜', na=False), 'keyword_han'] = True

neu_df['keyword_kmt'] = False
neu_df.loc[neu_df.content.str.contains('國民黨', na=False), 'keyword_kmt'] = True

neu_df['keyword_election'] = False
neu_df.loc[neu_df.content.str.contains('|'.join(['總統大選', '選舉', "大選"]), na=False), 'keyword_election'] = True


neu_df.head(2)


,level_0,index,Unnamed: 0,title,description,published date,url,publisher,content,source,keyword_tsai,keyword_dpp,keyword_han,keyword_kmt,keyword_election
0,0,0.0,0,蔡總統提名陳瑞敏為新任審計長 咨文送立法院| 政治 | 重點新聞 - 中央社即時新聞,蔡總統提名陳瑞敏為新任審計長 咨文送立法院| 政治 | 重點新聞 中央社即時新聞,"Mon, 26 Aug 2019 07:00:00 GMT",https://news.google.com/rss/articles/CBMiN2h0d...,"{'href': 'https://www.cna.com.tw', 'title': '中...",總統府表示，總統蔡英文提名現任行政院主計總處政務副主計長陳瑞敏為審計部審計長，咨請立法院行使...,cna,True,False,False,False,False
1,1,1.0,1,年輕人上第一線民進黨28日徵召吳怡農李問王閔生| 政治| 重點新聞 - 中央社即時新聞,年輕人上第一線民進黨28日徵召吳怡農李問王閔生| 政治| 重點新聞 中央社即時新聞,"Tue, 27 Aug 2019 07:00:00 GMT",https://news.google.com/rss/articles/CBMiN2h0d...,"{'href': 'https://www.cna.com.tw', 'title': '中...",民進黨提名策略小組今天拍板，徵召中研院學者吳乃德兒子吳怡農參選台北市中山北松山區立委，徵召市...,cna,True,True,False,True,True


In [17]:
neu_df = neu_df.drop(columns=['index', 'Unnamed: 0', 'description', 'url', 'publisher', 'level_0'])
neu_df.head(2)

,title,published date,content,source,keyword_tsai,keyword_dpp,keyword_han,keyword_kmt,keyword_election
0,蔡總統提名陳瑞敏為新任審計長 咨文送立法院| 政治 | 重點新聞 - 中央社即時新聞,"Mon, 26 Aug 2019 07:00:00 GMT",總統府表示，總統蔡英文提名現任行政院主計總處政務副主計長陳瑞敏為審計部審計長，咨請立法院行使...,cna,True,False,False,False,False
1,年輕人上第一線民進黨28日徵召吳怡農李問王閔生| 政治| 重點新聞 - 中央社即時新聞,"Tue, 27 Aug 2019 07:00:00 GMT",民進黨提名策略小組今天拍板，徵召中研院學者吳乃德兒子吳怡農參選台北市中山北松山區立委，徵召市...,cna,True,True,False,True,True


In [18]:
neu_df.sample(10)

,title,published date,content,source,keyword_tsai,keyword_dpp,keyword_han,keyword_kmt,keyword_election
1040,大銀台灣精銳等3台企加碼 根留台灣投資破244億 | 產經| 重點新聞 - 中央社即時新聞,"Thu, 24 Oct 2019 07:00:00 GMT",上銀旗下大銀微系統、台灣精銳科技及不具名光電大廠今天通過經濟部投資台灣事務所聯審，共將投資1...,cna,False,False,False,False,False
2089,前總統蔣介石生日忌 韓國瑜去慈湖謁陵 ｜ 公視新聞網 PNN - 公視新聞,"Thu, 31 Oct 2019 07:00:00 GMT",韓國瑜今仔日和明仔載兩工，攏佇桃園一帶咧拚選舉，因為今仔日是蔣介石前總統做生日忌，伊也代先去...,pts,False,False,True,True,True
217,蝮蛇Viper命名F-16V 國防部：接地氣說法| 政治 - 中央社即時新聞,"Thu, 05 Sep 2019 07:00:00 GMT",空軍向美採購F-16V戰機，針對取名「V」原因，國防部今天解釋，V是「Viper蝮蛇」之意，...,cna,False,False,False,False,False
1453,NCC發函中天三立新聞 籲回歸專業平衡報導| 政治 | 重點新聞 - 中央社即時新聞,"Wed, 30 Oct 2019 07:00:00 GMT",NCC今天召開委員會議，決議對三立新聞台「1000整點新聞」、「台灣大頭條」，以及中天新聞台...,cna,True,False,False,False,False
2080,傅達仁之子遵父遺願 積極推《尊嚴善終法》 ｜ 公視新聞網 PNN - 公視新聞,"Mon, 09 Sep 2019 07:00:00 GMT",知名體育主播傅達仁飽受胰臟癌所苦，去年遠赴瑞士、接受安樂死，而他的兒子傅俊豪今天以臉書直播、...,pts,False,False,False,True,False
1462,鐵路便當節復古風情39款口味邀民眾聞香品嚐| 生活 | 重點新聞 - 中央社即時新聞,"Tue, 29 Oct 2019 07:00:00 GMT",台鐵第5屆鐵路便當節11月1日開幕，在台北車站一樓多功能展演廳展出4天，並復刻40多年前台北...,cna,False,False,False,False,False
465,正隆砸20億高雄建廠擬進軍越南家用紙市場| 產經 - 中央社即時新聞,"Sun, 08 Sep 2019 07:00:00 GMT",正隆紙業今天舉行60週年嘉年華會，正隆總經理蔡東和表示，正隆現階段營運以越南為重心，平陽造紙...,cna,False,False,False,False,False
818,熊本熊想傳遞東奧聖火 因不是人類遭婉拒| 國際 | 重點新聞 - 中央社即時新聞,"Sat, 14 Dec 2019 08:00:00 GMT",日本東京奧運明年即將登場，以吉祥物「熊本熊」聞名的熊本縣政府，日前已向主辦單位詢問熊本熊擔任...,cna,False,False,False,False,False
325,張奕包辦12強投手3大獎 史上第一人| 運動 | 重點新聞 - 中央社即時新聞,"Mon, 18 Nov 2019 08:00:00 GMT",世界12強棒球賽落幕，日本隊奪下冠軍，中華隊則是第5名，不過張奕包辦投手3大獎，分別是全明星...,cna,False,False,False,False,False
1808,國民黨中常會場外爆衝突 青年突襲高喊老賊下台| 政治 | 重點新聞 - 中央社即時新聞,"Wed, 15 Jan 2020 08:00:00 GMT",國民黨今天下午召開中常會，前國民黨青年團總團長林家興率數名青年在黨部內抗議，高喊「常會道歉，...,cna,False,False,False,True,True


In [19]:
# neu_df[neu_df.source!="cna"]['source'] = 'reporter'
# print(neu_df.shape)
neu_df = neu_df.fillna({'source':'reporter'})
# only left the content that contain at least one of the keywords
neu_df = neu_df[neu_df[['keyword_tsai', 'keyword_han', 'keyword_kmt', 'keyword_dpp', 'keyword_election']].any(axis=1)]
print(neu_df.shape)
neu_df.groupby("source").count()

(962, 9)


,title,published date,content,keyword_tsai,keyword_dpp,keyword_han,keyword_kmt,keyword_election
source,,,,,,,,
cna,702,702,702,702,702,702,702,702
pts,144,144,144,144,144,144,144,144
reporter,116,116,116,116,116,116,116,116


In [20]:
neu_df['date'] = pd.to_datetime(neu_df["published date"], utc=True)
neu_df = neu_df.drop(columns=["published date"])
neu_df.sample(2)

,title,content,source,keyword_tsai,keyword_dpp,keyword_han,keyword_kmt,keyword_election,date
625,烏克蘭人民公僕陷貪腐醜聞 國會上演測謊秀| 國際 | 重點新聞 - 中央社即時新聞,上任不到半年的烏克蘭總統澤倫斯基以「打貪」作為主要競選口號之一，但他所創立「人民公僕黨」國會...,cna,False,False,False,False,True,2019-10-24 07:00:00+00:00
473,台南一片綠油油 洪秀柱挑戰王定宇掀藍綠大戰| 政治 | 重點新聞 - 中央社即時新聞,台南市向來是綠營票倉，自第7屆起5席立委全是民進黨籍，明年第10屆立委選舉受選區重劃影響，台...,cna,False,True,True,True,True,2019-09-14 07:00:00+00:00


In [21]:
neu_df['date'] = pd.to_datetime(neu_df.date)

In [22]:
neu_df['sub_source'] = neu_df.source
neu_df['source'] = "central"

## concat whole data

In [23]:
df = pd.concat([ltn_df, cti_df, neu_df]).reset_index(drop=True)
print(df.shape)

(11618, 11)


In [24]:
df

,title,keyword_dpp,keyword_han,keyword_kmt,keyword_tsai,index,content,keyword_election,date,source,sub_source
0,台中想吃雞排看這裡 2500份19、20日手刀排一波,False,False,False,True,6.0,館長兌現承諾發送雞排，其中2000份20日下午3點在陳柏惟烏日新興路481號服務處發放。（記...,True,2020-01-17 00:00:00+00:00,ltn,NaN
1,英國駐台代表晉見蔡總統 恭賀台灣蓬勃發展的民主機制,False,False,False,True,0.0,總統蔡英文（右）17日上午在總統府接見英國在台辦事處代表唐凱琳（Catherine Nett...,True,2020-01-17 00:00:00+00:00,ltn,NaN
2,與國外知名網紅聊女權、寵物 蔡總統：台灣重能力不看性別,False,False,False,True,1.0,總統蔡英文與百萬網紅塔米爾暢談各項議題。（中央社） 〔即時新聞／綜合報導〕總統蔡英文近期接受...,False,2020-01-17 00:00:00+00:00,ltn,NaN
3,酈英傑：美台盼締造共同未來 是「真朋友真進展」,False,False,False,True,2.0,美國在台協會與外交部17日舉辦「數位對話公共論壇：增進美台人民的關係」，美國在台協會處長酈英...,False,2020-01-17 00:00:00+00:00,ltn,NaN
4,台美瓜宏四方會談 吳釗燮：促進區域合作及發展,False,False,False,True,3.0,外交部長吳釗燮今在推特上推文表示，台美瓜宏舉行四方會談，推文照片並清楚可見四國國旗。（取自外...,False,2020-01-17 00:00:00+00:00,ltn,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11613,台中立委選戰 民進黨拿下5席、國民黨2席 ｜ 公視新聞網 PNN - 公視新聞,True,False,True,False,NaN,至於中台灣選...,True,2020-01-12 08:00:00+00:00,central,pts
11614,因慶富案虧損7.9億 高雄銀行告前老總背信 ｜ 公視新聞網 PNN - 公視新聞,False,False,False,False,NaN,慶富獵雷艦詐貸案是高雄銀行核貸給慶富17.4億元的履約保證金，造成後來虧損7.9億，高銀內部...,True,2020-01-07 08:00:00+00:00,central,pts
11615,總統：台灣已是獨立國家 叫「中華民國台灣」 ｜ 公視新聞網 PNN - 公視新聞,False,False,False,True,NaN,2020大選...,True,2020-01-15 08:00:00+00:00,central,pts
11616,We are an Independent Country Already: Preside...,False,False,False,True,NaN,President Tsai Ing-wen has secured a second te...,True,2020-01-15 08:00:00+00:00,central,pts


In [25]:
df.to_csv(path+ 'TW_news.csv', index=False )